# 최소 신장 트리 (Minimum Spanning Tree)

## 최소 스패닝 트리

- 문제 출처: [백준 1197번](https://www.acmicpc.net/problem/1197)

`-` `Kruskal Algorithm`을 사용하여 문제를 해결하겠다(`Prim Algorithm`도 있음!!)

`step 1` w(노드 u와 노드v 사이 간선의 가중치)를 기준으로 (u, v, w)를 오름차순 정렬한다

`step 2` 가장 작은 간선 가중치를 가지는 (노드 u, 노드 v)를 pop하고 u와 v가 연결됐을 때 MST가 사이클을 형성하지 않는다면 u와 v를 연결 

`step 3` spanning tree가 완성될 때까지 `step 2`를 반복 ---> 완성된 spanning tree는 minimum spanning tree (MST)

In [2]:
import sys

sys.setrecursionlimit(10**6)


# cycle여부를 판단하기 위한 disjoint-set
def make_set(u):
    p[u] = u  # 각 노드가 자기자신을 가리키게 한다 (u -> u)
    rank[u] = 0


def find_set(u):  # u가 포함된 tree의 부모 노드를 찾아준다
    if p[u] != u:  # u가 자기자신을 가리키지 않으면 (=자식 노드)
        p[u] = find_set(p[u])  # flatten tree, original: (1 -> 3, 3 -> 5, 5 -> 7, 7 -> 7), new: (1 -> 7, 3-> 7, 5 -> 7, 7 -> 7)
    return p[u]


def union_set(u, v):
    uu = find_set(u)
    vv = find_set(v)
    if uu == vv:  # uu와 vv가 같다면 이미 같은 tree에 속하므로 union할 이유가 없다
        return
    rank_u = rank[uu]
    rank_v = rank[vv]
    if rank_u > rank_v:  # v -> u
        p[vv] = uu
    elif rank_u == rank_v:  # v -> u(u -> v도 가능) and rank에 +1
        p[vv] = uu
        rank[vv] += 1
    else:  # u -> v
        p[uu] = vv


def kruskal(graph):
    mst_cost = 0
    for w, u, v in graph: 
        if find_set(u) == find_set(v):
            continue
        # cycle을 형성하지 않는다면
        union_set(u, v)
        mst_cost += w
    return mst_cost


def prepare_graph(E):
    graph = []
    for _ in range(E):
        u, v, w = map(int, input().split())
        graph.append([w, u, v]) 
    graph.sort()  # 가중치를 기준으로 오름차순 정렬
    return graph


def solution():
    global p, rank
    V, E = map(int, input().split())
    p = [0 for _ in range(V + 1)]  # node[i]는 i번 노드가 가리키는(point) 노드를 나타냄
    rank = [0 for _ in range(V + 1)]  # rank[i]는 i번 노드의 rank 상한을 나타냄  
    graph = prepare_graph(E)
    for i in range(1, V + 1):
        make_set(i)
    mst_cost = kruskal(graph)
    print(mst_cost)


solution()

# input
# 3 3
# 1 2 1
# 2 3 2
# 1 3 3

 3 3
 1 2 1
 2 3 2
 1 3 3


3


`-` 왜 올바르게 동작하는지는 다음 링크 참고: https://en.wikipedia.org/wiki/Kruskal%27s_algorithm#Proof_of_correctness

## 도시 분할 계획

- 문제 출처: [백준 1647번](https://www.acmicpc.net/problem/1647)

`-` 최소 신장 트리를 만든 후 간선 중 최댓값을 제거하면 최소 비용으로 두 개의 마을로 분리할 수 있다

`-` 이게 성립하는 이유를 간단히 알아보자

`-` 일단, 정답에 해당하는 마을 $a,b$를 고려하자

`-` 이 마을 $a,b$는 두 개의 분리된 마을로 가능한 모든 경우 중 최소 비용을 가진다

`-` 임의의 두 집 사이에는 경로가 항상 존재한다고 했다

`-` 그럼 $a$ 마을에서 집 $h_a$를 고르고 $b$ 마을에서 집 $h_b$를 고를 수 있다

`-` $h_a$와 $h_b$를 잇는 최소 비용의 길 $r$을 골라 이으면 전체 마을은 이어지게 되며 이는 최소 신장 트리여야 한다

`-` 만약 이게 최소 신장 트리가 아니라고 해보자

`-` $a$ 마을에서 $r$과 연결된 집 $h_b$까지 포함해 새로운 마을 $c$를 만든다고 해보자 (집 $h_b$는 그래프 상에서 끝 점이다 = 연결된 점이 하나이다)

`-` 마을 $c$는 최소 신장 트리여야한다

`-` 마을 $c$가 최소 신장 트리가 아니라고 가정하자

`-` 마을 $a$와 집 $h_b$는 최소 비용으로 연결되어 있으므로 더 나은 경로는 없다

`-` 마을 $a$도 최소 신장 트리이므로 이보다 더 적은 비용으로 마을 $a$를 재구성할 수 없다

`-` 따라서 마을 $c$가 최소 신장 트리가 아닐 수 없다

`-` 이런 방식으로 마을 $b$의 모든 집을 포함시킬 수 있고 이들은 항상 최소 신장 트리이다

`-` 따라서 정답에 해당하는 마을 $a$와 마을 $b$를 최소 비용으로 연결해 생긴 전체 마을은 최소 신장 트리이다

`-` 즉, 전체 마을에 대해 최소 신장 트리를 만들고 최대 비용 간선을 제거하면 마을 $a$와 $b$로 분리된다

`-` 최대 비용 간선을 제거하지 않으면 더 적은 비용으로 두 개의 마을로 나눌 수 있으므로 최대 비용 간선을 제거해야 정답 마을 $a$와 $b$가 된다

`-` 최소 신장 트리는 크루스칼 알고리즘으로 구현하자

`-` 그래프 상에서 최소 비용으로 연결된 두 간선을 사이클이 생기지 않는 선에서 합쳐나가면 된다

- 최소 신장 트리를 나누면 각 그룹도 최소 신장 트리인 이유

`-` 일단 최소 신장 트리의 임의의 간선을 제거하여 두 그룹으로 나누면 각 그룹은 최소 신장 트리이다

`-` 만약 각 그룹이 최소 신장 트리가 아니라면 더 적은 비용으로 각 그룹을 재구성 할 수 있고 이 둘을 연결하면 원래보다 비용이 더 적어진다

`-` 그런데 처음에 최소 신장 트리라고 한 가정과 모순되므로 각 그룹은 최소 신장 트리여야 한다

In [3]:
import heapq


def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if u != p[u]:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return False
    if rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    elif rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    else:
        p[u_root] = v_root
        rank[v_root] += 1
    return True


def kruskal(graph):
    count = 0
    mst_cost = 0
    max_edge_weight = 0
    while count < N - 1:
        cost, a, b = heapq.heappop(graph)
        if not union(a, b):
            continue
        mst_cost += cost
        count += 1
        max_edge_weight_in_mst = max(cost, max_edge_weight)
    return mst_cost, max_edge_weight_in_mst


def solution():
    global N, p, rank
    N, M = map(int, input().split())
    p = [i for i in range(N + 1)]
    rank = [0 for _ in range(N + 1)]
    for i in range(1, N + 1):
        make_set(i)
    graph = []
    for _ in range(M):
        A, B, C = map(int, input().split())
        graph.append((C, A, B))
    heapq.heapify(graph)
    mst_cost, max_edge_weight_in_mst = kruskal(graph)
    print(mst_cost - max_edge_weight_in_mst)


solution()

# input
# 7 12
# 1 2 3
# 1 3 2
# 3 2 1
# 2 5 2
# 3 4 4
# 7 3 6
# 5 1 5
# 1 6 2
# 6 4 1
# 6 5 3
# 4 5 3
# 6 7 4

 7 12
 1 2 3
 1 3 2
 3 2 1
 2 5 2
 3 4 4
 7 3 6
 5 1 5
 1 6 2
 6 4 1
 6 5 3
 4 5 3
 6 7 4


8


## 별자리 만들기

- 문제 출처: [백준 4386번](https://www.acmicpc.net/problem/4386)

`-` 임의의 두 점 사이의 거리(거리 행렬)을 계산하는 로직의 시간 복잡도는 $O\left(N^2\right)$이다

`-` 그래프가 만들어졌으므로 크루스칼 알고리즘을 사용해 최소 신장 트리의 비용을 계산하자

In [19]:
def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    else:
        p[u_root] = v_root
        rank[u_root] += 1


def make_distance_matrix(positions, n):
    graph = []
    for i in range(n):
        x_i, y_i = positions[i]
        for j in range(i + 1, n):
            x_j, y_j = positions[j]
            dist = ((x_i - x_j)**2 + (y_i - y_j)**2)**0.5
            graph.append((dist, i, j))
    return graph    


def kruskal(graph):
    mst_cost = 0
    for dist, u, v in graph:
        if find(u) == find(v):
            continue
        union(u, v)
        mst_cost += dist
    return mst_cost


def solution():
    global p, rank
    n = int(input())
    p = [i for i in range(n)]
    rank = [0 for _ in range(n)]
    positions = []
    for _ in range(n):
        x, y = map(float, input().split())
        positions.append((x, y))
    for u in range(n):
        make_set(u)
    graph = make_distance_matrix(positions, n)
    graph.sort()
    mst_cost = kruskal(graph)
    print(mst_cost)


solution()

# input
# 3
# 1.0 1.0
# 2.0 2.0
# 2.0 4.0

 3
 1.0 1.0
 2.0 2.0
 2.0 4.0


3.414213562373095


## 우주신과의 교감

- 문제 출처: [백준 1774번](https://www.acmicpc.net/problem/1774)

`-` [별자리 만들기](https://www.acmicpc.net/problem/4386) 문제와 비슷하게 해결할 수 있다

`-` 두 우주신을 잇는 비용을 알아냐 하니 거리 행렬을 만들자

`-` 이미 연결된 통로에 대해 합집합 연산을 해두자 (새로 만들 통로의 길이 합이 최소가 되기만 하면 되므로 이미 연결된 통로의 비용은 고려하지 않아도 된다)

`-` 이 상태에서 크루스칼 알고리즘을 사용해 최소 신장 트리를 만들면 된다

`-` 소수점 둘째 자리까지 반올림하여 출력해야 하는데 파이썬의 `round` 함수는 일반적인 반올림은 아니다

`-` `round` 함수를 사용해 $3.5$를 반올림하면 $4$이지만 $4.5$를 반올림하면 $4$이다

`-` 커스텀 반올림 함수를 만들어 해결하자

`-` 출력 부분을 소수점 둘째 자리까지 나오도록 변경하니 맞았다!

`-` `3.855` 같은 입력은 없었나보다

In [148]:
def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    else:
        p[u_root] = v_root
        rank[u_root] += 1


def make_distance_matrix(positions, n):
    n = len(positions)
    graph = []
    for i in range(n):
        x_i, y_i = positions[i]
        for j in range(i + 1, n):
            x_j, y_j = positions[j]
            dist = ((x_i - x_j)**2 + (y_i - y_j)**2)**0.5
            graph.append((dist, i, j))
    return graph    


def kruskal(graph):
    mst_cost = 0
    for dist, u, v in graph:
        if find(u) == find(v):
            continue
        union(u, v)
        mst_cost += dist
    return mst_cost


def solution():
    global p, rank
    N, M = map(int, input().split())
    p = [i for i in range(N)]
    rank = [0 for _ in range(N)]
    positions = []
    for _ in range(N):
        x, y = map(int, input().split())
        positions.append((x, y))
    for u in range(N):
        make_set(u)
    for _ in range(M):
        u, v = map(lambda x: int(x) - 1, input().split())
        union(u, v)
    graph = make_distance_matrix(positions, N)
    graph.sort()
    mst_cost = kruskal(graph)
    print(f"{mst_cost:.2f}")


solution()

# input
# 4 1
# 1 1
# 3 1
# 2 3
# 4 3
# 1 4

 4 1
 1 1
 3 1
 2 3
 4 3
 1 4


4.00


## 전력난

- 문제 출처: [백준 6497번](https://www.acmicpc.net/problem/6497)

`-` 간단한 최소 신장 트리 문제이다

`-` 두 집 쌍과 거리를 그래프에 기록한다

`-` 그래프상의 거리들의 합을 계산한다

`-` 크루스칼 알고리즘을 통해 최소 신장 트리의 비용을 계산한다

`-` 전체 거리들의 합에서 최소 신장 트리의 비용를 제외하면 절약할 수 있는 최대 비용이다

In [3]:
import sys

sys.setrecursionlimit(10**6)


def make_set(u):
    p[u] = u
    rank[u] = 0


def find(u):
    if p[u] != u:
        p[u] = find(p[u])
    return p[u]


def union(u, v):
    u_root = find(u)
    v_root = find(v)
    if u_root == v_root:
        return
    if rank[v_root] < rank[u_root]:
        p[v_root] = u_root
    elif rank[u_root] < rank[v_root]:
        p[u_root] = v_root
    else:
        p[u_root] = v_root
        rank[u_root] += 1


def kruskal(graph):
    mst_cost = 0
    for dist, u, v in graph:
        if find(u) == find(v):
            continue
        union(u, v)
        mst_cost += dist
    return mst_cost


def solve_testcase(m, n):
    global p, rank
    p = [i for i in range(m)]
    rank = [0 for _ in range(m)]
    graph = []
    total_cost = 0
    for _ in range(n):
        x, y, cost = map(int, input().split())
        total_cost += cost
        graph.append((cost, x, y))
    for u in range(m):
        make_set(u)
    graph.sort()
    mst_cost = kruskal(graph)
    answer = total_cost - mst_cost
    print(answer)


def solution():
    while True:
        m, n = map(int, input().split())
        if m == 0 and n == 0:
            break
        solve_testcase(m, n)


solution()

# input
# 7 11
# 0 1 7
# 0 3 5
# 1 2 8
# 1 3 9
# 1 4 7
# 2 4 5
# 3 4 15
# 3 5 6
# 4 5 8
# 4 6 9
# 5 6 11
# 0 0

 7 11
 0 1 7
 0 3 5
 1 2 8
 1 3 9
 1 4 7
 2 4 5
 3 4 15
 3 5 6
 4 5 8
 4 6 9
 5 6 11


51


 0 0
